In [3]:
import torch

In [2]:
# #add neobert to kernel PATH
# import sys
# import os
# parent_dir = os.path.abspath('..')
# if parent_dir not in sys.path:

#     sys.path.append (parent_dir) 

In [4]:
import sys
sys.path.append("C:/Users/gruau/OneDrive/Documents/CentraleSupelec/3A/Stage Oxford/Implementation/NeoBERT/NeoBERT_dev/src/")


In [5]:
import importlib
import neobert.model as mdl
importlib.reload(mdl)

#importlib.reload(mdl)

c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\NeoBERTenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'neobert.model' from 'C:\\Users\\gruau\\OneDrive\\Documents\\CentraleSupelec\\3A\\Stage Oxford\\Implementation\\NeoBERT\\NeoBERT_dev\\src\\neobert\\model\\__init__.py'>

In [ ]:
import neobert.model as mdl  # or your actual import if different

# Create a minimal config
config = mdl.NeoBERTConfig(
    vocab_size=100,            # must be > max index in dummy_input
    hidden_size=8,             # divisible by num_attention_heads
    num_attention_heads=2,     # divides hidden_size evenly
    num_hidden_layers=2,       # small number of layers
    intermediate_size=16,      # FFN dim
    max_length=16,             # match seq_len
    pad_token_id=0,            # define padding token index
    rope=False, 
    expert_sizes = '0,1152, 1408, 1664, 1920, 2176, 2432, 2688, 2944',              # simpler
    flash_attention=False      # simpler
)

# Instantiate model
model = mdl.NeoBERTLMHead(config)

# Generate dummy input (ensure all token indices < vocab_size)
batch_size = 2
seq_len = 16

# Token indices from 1 to vocab_size-1 (reserve 0 for pad_token)
dummy_input = torch.randint(1, config.vocab_size, (batch_size, seq_len))

# Add some padding tokens at the end (optional)
dummy_input[:, -2:] = config.pad_token_id  # last 2 tokens are padding

# Generate additive pad mask: 0.0 for valid tokens, -inf for padding
pad_mask = (dummy_input != config.pad_token_id).float()
pad_mask = torch.where(pad_mask == 1, 0.0, float('-inf'))

# Run forward pass
model.eval()
with torch.no_grad():
    output = model(dummy_input, pad_mask=pad_mask)

# Check outputs
print("Logits shape:", output["logits"].shape)
print("Expert usage loss:", output["expert_usage_loss"])


W0730 10:54:34.700000 46140 Lib\site-packages\torch\_dynamo\convert_frame.py:1125] WON'T CONVERT forward C:\Users/gruau/OneDrive/Documents/CentraleSupelec/3A/Stage Oxford/Implementation/NeoBERT/NeoBERT_dev/src\neobert\model\rmsnorm.py line 23 
W0730 10:54:34.700000 46140 Lib\site-packages\torch\_dynamo\convert_frame.py:1125] due to: 
W0730 10:54:34.700000 46140 Lib\site-packages\torch\_dynamo\convert_frame.py:1125] Traceback (most recent call last):
W0730 10:54:34.700000 46140 Lib\site-packages\torch\_dynamo\convert_frame.py:1125]   File "c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\neoBERT_env\Lib\site-packages\torch\_inductor\cpp_builder.py", line 130, in check_compiler_exist_windows
W0730 10:54:34.700000 46140 Lib\site-packages\torch\_dynamo\convert_frame.py:1125]     subprocess.check_output([compiler, "/help"], stderr=subprocess.STDOUT)
W0730 10:54:34.700000 46140 Lib\site-packages\torch\_dynamo\convert_frame.py:1125]   File "C:\Users\gru

Logits shape: torch.Size([2, 16, 100])
Expert usage loss: tensor(7864528.5000)


Look into the architecture

In [144]:
import neobert.model as mdl  # or your actual import if different

# Create a minimal config
config = mdl.NeoBERTConfig(
)
# Instantiate model
model = mdl.NeoBERTLMHead(config)


sd = model.state_dict()
for k,v in sd.items(): 
    print(k,v.shape)

model.encoder.weight torch.Size([32768, 768])
model.transformer_encoder.0.qkv.weight torch.Size([2304, 768])
model.transformer_encoder.0.wo.weight torch.Size([768, 768])
model.transformer_encoder.0.attention_norm.weight torch.Size([768])
model.transformer_encoder.0.expert_norm.weight torch.Size([768])
model.transformer_encoder.0.block_moe.gate.weight torch.Size([3, 768])
model.transformer_encoder.0.block_moe.experts.1.ffn.in_proj.weight torch.Size([2048, 768])
model.transformer_encoder.0.block_moe.experts.1.ffn.out_proj.weight torch.Size([768, 1024])
model.transformer_encoder.0.block_moe.experts.2.ffn.in_proj.weight torch.Size([4096, 768])
model.transformer_encoder.0.block_moe.experts.2.ffn.out_proj.weight torch.Size([768, 2048])
model.transformer_encoder.1.qkv.weight torch.Size([2304, 768])
model.transformer_encoder.1.wo.weight torch.Size([768, 768])
model.transformer_encoder.1.attention_norm.weight torch.Size([768])
model.transformer_encoder.1.expert_norm.weight torch.Size([768])
mod

In [ ]:
def get_expert_mask(gate_logits,cfg):
    if isinstance(gate_logits, tuple):
        compute_device = gate_logits[0].device
        stacked_gate_logits = torch.stack([layer_gate.to(compute_device) for layer_gate in gate_logits], dim=0)#n_layers, batch_size*seq_length, n_experts
        _,_, num_experts = stacked_gate_logits.shape
    routing_weights = torch.nn.functional.softmax(stacked_gate_logits, dim=-1)


    if cfg.model.routing_strategy == "top_k":
        top_k = cfg.model.num_experts_per_tok_inference #if we wanted  to use top_k for heterogeneous moe
        _, selected_experts = torch.topk(routing_weights, top_k, dim=-1)
        target_expert_mask = torch.nn.functional.one_hot(selected_experts, num_experts).sum(2)#n_layers, batch_size*seq_length, n_experts
    
    elif cfg.model.routing_strategy == "top_p":
        top_p = cfg.model.min_expert_cumprob_per_token
        sorted_weights, sorted_indices = torch.sort(routing_weights, dim=-1, descending=False)

        cum_probs = sorted_weights.cumsum(dim=-1)
        mask = cum_probs > 1 - top_p

        unsorted_mask = torch.zeros_like(mask, dtype=torch.bool)
        target_expert_mask = unsorted_mask.scatter(dim=-1, index=sorted_indices, src=mask) #n_layers, batch_size*seq_length, n_experts

    return target_expert_mask # n_layers, batch_size*n_seq, n_experts


neobert_model_output = neobert_model(batch["input_ids"], batch.get("attention_mask", None), output_expert_usage_loss=False, output_router_logits=True)
target_gate_logits = neobert_model_output['router_logits'] #tuple  of n_layers of batch_size*seq_length,  n_experts

target_expert_mask = get_expert_mask(target_gate_logits, cfg) # n_layers, batch_size*n_seq, n_experts

predicted_routed_logits = compiled_BERTpredictor(input_ids=batch["input_ids"])
predicted_routed_logits = torch.stack(predicted_routed_logits, dim=0) #n_layers, batch_size*seq_length, n_experts

#ignore padded tokens
pad_mask = batch.get("attention_mask", None)
if pad_mask is not None:
    pad_mask = pad_mask.view(-1, 1)
    pad_mask = (pad_mask != float("-inf")).squeeze(-1)
    predicted_routed_logits = predicted_routed_logits[:,pad_mask,:]
    target_expert_mask = target_expert_mask[:,pad_mask,:]

#reshape 
predicted_routed_logits = predicted_routed_logits.view(-1, predicted_routed_logits.size(-1)) #n_layers*batch_size*seq_length, n_experts
target_expert_mask = target_expert_mask.view(-1, target_expert_mask.size(-1)) #n_layers*batch_size*seq_length, n_experts

# Under the no_sync context manager, PyTorch will skip synchronizing the gradients when .backward() is
# called, and the first call to .backward() outside this context manager will trigger the synchronization.
# Accumulating manually gives more flexibility and is compatible with TPUs.
# if metrics["train/batches"] % cfg.trainer.gradient_accumulation_steps != 0:
#     with accelerator.no_sync(BERTpredictor):
#         loss = loss_fn(predicted_routed_logits, target_expert_mask*1.0)
#         loss.backward()

loss = loss_fn(predicted_routed_logits, target_expert_mask*1.0)
accelerator.backward(loss)


In [43]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])

b = torch.tensor([[1, 2, 3], [4, 5, 6]])

c = ()
d = c + (a,)+ (b,)
# print(type(d))
# print(d)

e = torch.cat([tens for tens in d], dim=0)

print(e.shape)
print(e)

f = torch.nn.functional.softmax(e.float(), dim=-1)
print(f.shape)
print(f)


entropy = torch.sum(-f * torch.log(f + 1e-10), dim=-1)
print(entropy.shape)
print(entropy)
# g = torch.mean(f, dim=0)
# # print(g.shape)
# # print(g)
# _, selected_experts = torch.topk(f, 2, dim=-1)

# expert_mask = torch.nn.functional.one_hot(selected_experts, 3)
# expert_size = [int(expert_size) for expert_size in '2,4,6'.split(',')]
# print(expert_size)
# expert_mask = expert_mask *torch.tensor(expert_size)
# print(expert_mask.shape)
# print(expert_mask)

torch.Size([4, 3])
tensor([[1, 2, 3],
        [4, 5, 6],
        [1, 2, 3],
        [4, 5, 6]])
torch.Size([4, 3])
tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])
torch.Size([4])
tensor([0.8324, 0.8324, 0.8324, 0.8324])


In [39]:
len('a,b,c'.split(','))

3

In [ ]:
x = torch.tensor([10, 9, 7, 4, 2], dtype=torch.float)
p = 20.0

# Step 1: Cumulative sum
cumsum = torch.cumsum(x, dim=0)

# Step 2: Find first index where cumsum >= p
 

print(mask)

tensor([1, 1, 1, 0, 0], dtype=torch.uint8)


In [47]:
sorted_indices = torch.tensor([0, 1, 2, 3, 4])
sorted_indices[mask]

C:\Users\gruau\AppData\Local\Temp\ipykernel_35176\2055800616.py:2: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:28.)
  sorted_indices[mask]


tensor([0, 1, 2])

In [ ]:
num_params = sum(p.numel() for p in model.parameters())
print(num_params)


544842900


In [ ]:
k 

print("mask", mask)
masked_weights = mask*routing_weights


print("expert hittd",expert_hitted)
print(expert_hitted_weight)


tensor([[0.3907, 0.2894, 0.3199],
        [0.3006, 0.3322, 0.3672]])
tensor([[0.2894, 0.3199, 0.3907],
        [0.3006, 0.3322, 0.3672]]) tensor([[1, 2, 0],
        [0, 1, 2]])
mask tensor([[False,  True,  True],
        [False,  True,  True]])
tensor([[0.0000, 0.2894, 0.3199],
        [0.0000, 0.3322, 0.3672]])
expert hittd tensor([2, 0, 1, 2])
tensor([0.3199, 0.3907, 0.3322, 0.3672])


In [97]:
top_p = 0.4

# Step 1: Softmax routing weights
routing_weights = torch.nn.functional.softmax(torch.tensor([[0.5, 0.2, 0.3], [0.4, 0.5, 0.6],[0.5, 0.1, 0.3], [0.2, 0.5, 0.6]], dtype=torch.float), dim=-1)
print("Routing weights:\n", routing_weights)


routing_weights, selected_experts = torch.topk(routing_weights, top_k, dim=-1)

num_experts = 3
# One hot encode the selected experts to create an expert mask
# this will be used to easily index which expert is going to be sollicitated
expert_mask = torch.nn.functional.one_hot(selected_experts, num_classes=num_experts).permute(2, 1, 0)
print(expert_mask)
expert_hitted = torch.greater(expert_mask.sum(dim=(-1, -2)), 0).nonzero()
print("Expert hitted:", expert_hitted)


Routing weights:
 tensor([[0.3907, 0.2894, 0.3199],
        [0.3006, 0.3322, 0.3672],
        [0.4018, 0.2693, 0.3289],
        [0.2603, 0.3514, 0.3883]])
tensor([[[1, 0, 1, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 1, 0, 1]],

        [[0, 1, 0, 1],
         [1, 0, 1, 0]]])
Expert hitted: tensor([[0],
        [1],
        [2]])


In [125]:
import torch

top_p = 0.4
batch_size = 1; sequence_length = 4; hidden_dim = 2
# Step 1: Softmax routing weights
routing_weights = torch.nn.functional.softmax(torch.tensor([[0.5, 0.2, 0.3], [0.4, 0.5, 0.6],[0.5, 0.1, 0.3], [0.2, 0.5, 0.6]], dtype=torch.float), dim=-1)
hidden_states = torch.randn(batch_size*sequence_length,hidden_dim)  # Simulated current state for demonstration
print("Routing weights:\n", routing_weights)
print("hidden states:\n", hidden_states)

# Step 2: Sort weights and indices
sorted_weights, sorted_indices = torch.sort(routing_weights, dim=-1, descending=False)
print("Sorted weights:\n", sorted_weights)
print("Sorted indices:\n", sorted_indices)

# Step 3: Compute cumulative probabilities
cum_probs = sorted_weights.cumsum(dim=-1)
# Step 4: Mask for top-p
mask = cum_probs > 1 - top_p  # same shape as routing_weights
unsorted_mask = torch.zeros_like(mask, dtype=mask.dtype)
expert_mask = unsorted_mask.scatter_(dim=-1, index=sorted_indices, src=mask).permute(1, 0)
print("expert_mask", expert_mask)
expert_hitted = torch.greater(expert_mask.sum(dim=(-1)), 0).nonzero()
for expert_idx in expert_hitted:
    token_idx= torch.where(expert_mask[expert_idx].squeeze(0))
    print(idx)
    current_state = hidden_states[None,token_idx].reshape(-1, hidden_dim)
    print(current_state)
    #current_hidden_states = expert_layer(current_state) * routing_weights[token_idx, expert_idx, None]

   






Routing weights:
 tensor([[0.3907, 0.2894, 0.3199],
        [0.3006, 0.3322, 0.3672],
        [0.4018, 0.2693, 0.3289],
        [0.2603, 0.3514, 0.3883]])
hidden states:
 tensor([[-0.1232, -0.7873],
        [-0.1052, -0.0381],
        [-0.0502,  2.0465],
        [-0.9725,  0.6955]])
Sorted weights:
 tensor([[0.2894, 0.3199, 0.3907],
        [0.3006, 0.3322, 0.3672],
        [0.2693, 0.3289, 0.4018],
        [0.2603, 0.3514, 0.3883]])
Sorted indices:
 tensor([[1, 2, 0],
        [0, 1, 2],
        [1, 2, 0],
        [0, 1, 2]])
expert_mask tensor([[ True, False,  True, False],
        [False,  True, False,  True],
        [ True,  True, False,  True]])
(tensor([0, 2]),)


TypeError: only integer tensors of a single element can be converted to an index

In [153]:
hidden_states = torch.randn(batch_size * sequence_length, hidden_dim)
# Dummy expert modules (simulate expert layers)
class DummyExpert(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        return self.linear(x)

experts = torch.nn.ModuleList([DummyExpert() for _ in range(num_experts)])

In [ ]:
import torch
import torch.nn.functional as F

top_p = 0.4
num_experts = 3
batch_size = 1
sequence_length = 4
hidden_dim = 2



# Routing weights and hidden states
routing_logits = torch.tensor([
    [0.5, 0.2, 0.3],
    [0.4, 0.5, 0.6],
    [0.5, 0.1, 0.3],
    [0.2, 0.5, 0.6]
], dtype=torch.float)

routing_weights = F.softmax(routing_logits, dim=-1)


# Sort and compute cumulative probability


print("Final routed hidden states:\n", final_hidden_states)


tensor([[0],
        [1],
        [2]])
Final routed hidden states:
 tensor([[ 0.1863, -0.1310],
        [ 0.1626, -0.4599],
        [ 0.2274, -0.0569],
        [ 0.5481,  0.0104]], grad_fn=<IndexAddBackward0>)


In [4]:
import torch

top_p = 0.7

routing_weights = torch.nn.functional.softmax(
    torch.tensor([[0.9, 0.8, 0.1,0.1,0.2], [0.1, 0.1, 0.2,0.1,0.2]], dtype=torch.float), dim=-1)

# Sort the weights to prepare for cumulative probability
sorted_weights, sorted_indices = torch.sort(routing_weights, dim=-1, descending=False)
cum_probs = sorted_weights.cumsum(dim=-1)

# Build mask for top-p selection
mask = cum_probs > 1 - top_p

# Apply mask to sorted_indices
# Get expert indices per token based on top-p mask
expert_indices_per_token = []
max_selected = 0  # to determine padding width

for token_mask, token_sorted_indices in zip(mask, sorted_indices):
    selected = token_sorted_indices[token_mask]
    expert_indices_per_token.append(selected)
    max_selected = max(max_selected, selected.numel())

# Pad to build final tensor
padded_indices = torch.full((len(expert_indices_per_token), max_selected), -1, dtype=torch.long)

for i, selected in enumerate(expert_indices_per_token):
    padded_indices[i, :selected.numel()] = selected

print("Final tensor of expert indices per token (padded):")
print(padded_indices)


Final tensor of expert indices per token (padded):
tensor([[ 4,  1,  0, -1],
        [ 1,  3,  2,  4]])


In [ ]:
import torch

top_p = 0.4

routing_weights = torch.nn.functional.softmax(
    torch.tensor([[0.5, 0.2, 0.3], [0.4, 0.5, 0.6]], dtype=torch.float), dim=-1)

# Sort the weights to prepare for cumulative probability
sorted_weights, sorted_indices = torch.sort(routing_weights, dim=-1, descending=False)
cum_probs = sorted_weights.cumsum(dim=-1)

# Build mask for top-p selection
mask = cum_probs > 1 - top_p

# Apply mask to sorted_indices
# Get expert indices per token based on top-p mask
expert_indices_per_token = []
max_selected = 0  # to determine padding width

for token_mask, token_sorted_indices in zip(mask, sorted_indices):
    selected = token_sorted_indices[token_mask]
    expert_indices_per_token.append(selected)
    max_selected = max(max_selected, selected.numel())

# Pad to build final tensor
padded_indices = torch.full((len(expert_indices_per_token), max_selected), -1, dtype=torch.long)

for i, selected in enumerate(expert_indices_per_token):
    padded_indices[i, :selected.numel()] = selected

print("Final tensor of expert indices per token (padded):")
print(padded_indices)


In [67]:
top_k = 2
routing_weights, selected_experts = torch.topk(routing_weights, top_k, dim=-1)
print(routing_weights, selected_experts)

tensor([[0.3907, 0.3199],
        [0.3672, 0.3322]]) tensor([[0, 1],
        [0, 1]])


In [2]:
import sys
print("\n".join(sys.path))

C:\Users\gruau\anaconda3\python312.zip
C:\Users\gruau\anaconda3\DLLs
C:\Users\gruau\anaconda3\Lib
C:\Users\gruau\anaconda3
c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\neoBERT_env

c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\neoBERT_env\Lib\site-packages
c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\neoBERT_env\Lib\site-packages\win32
c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\neoBERT_env\Lib\site-packages\win32\lib
c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\neoBERT_env\Lib\site-packages\Pythonwin
C:/Users/gruau/OneDrive/Documents/CentraleSupelec/3A/Stage Oxford/Implementation/NeoBERT/NeoBERT_dev/src/


Test top k routing/ NEED TO add MoeBLock to the init 

In [10]:
import importlib
import neobert.model as mdl
importlib.reload(mdl)

#importlib.reload(mdl)

<module 'neobert.model' from 'C:\\Users\\gruau\\OneDrive\\Documents\\CentraleSupelec\\3A\\Stage Oxford\\Implementation\\NeoBERT\\NeoBERT_dev\\src\\neobert\\model\\__init__.py'>

In [8]:

# Create a minimal config
config = mdl.NeoBERTConfig(
    vocab_size=100,            # must be > max index in dummy_input
    hidden_size=2,             # divisible by num_attention_heads
    num_attention_heads=2,     # divides hidden_size evenly
    num_hidden_layers=2,       # small number of layers
    intermediate_size=16,      # FFN dim
    max_length=16,             # match seq_len
    pad_token_id=0,
    routing_strategy="top_k",
    num_experts_per_tok_inference=1,                                       # define padding token index
    rope=False,                # simpler
    flash_attention=False      # simpler
)

# Instantiate mode

# Generate dummy input (ensure all token indices < vocab_si14ze)
batch_size = 1
seq_len = 4

test_input = torch.randn(batch_size,seq_len,config.hidden_size)
test_input_shape = test_input.shape
print("test_input: ", test_input)

MoeBlock = mdl.MoEBlock(config)
MoeBlock.eval()

MoeBlock(hidden_states = test_input, output_expert_usage_loss = False)


test_input:  tensor([[[-0.1565,  0.0036],
         [-0.8491,  1.5031],
         [-1.1938, -0.4445],
         [ 0.0880, -0.1160]]])
rtouter_logits tensor([[-0.0743,  0.6403, -0.4663],
        [ 0.0646, -0.5416, -0.9011],
        [ 0.1294,  0.7737, -0.2321],
        [-0.1226,  0.7680, -0.4474]], grad_fn=<AddmmBackward0>)
hidden_states tensor([[-0.1565,  0.0036],
        [-0.8491,  1.5031],
        [-1.1938, -0.4445],
        [ 0.0880, -0.1160]])
routing_weights tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<DivBackward0>)
expert_idx tensor([0])
expert_layer Expert()
current_hidden_states tensor([[-0.8491,  1.5031]], grad_fn=<MulBackward0>)
expert_idx tensor([1])
expert_layer Expert(
  (ffn): SwiGLU(
    (in_proj): Linear(in_features=2, out_features=2048, bias=False)
    (out_proj): Linear(in_features=1024, out_features=2, bias=False)
  )
)
current_hidden_states tensor([[1.0416e-04, 2.0962e-04],
        [2.2493e-03, 4.1346e-02],
        [4.9990e-04, 8.1566e-06]], grad_f

(tensor([[[ 1.0416e-04,  2.0962e-04],
          [-8.4912e-01,  1.5031e+00],
          [ 2.2493e-03,  4.1346e-02],
          [ 4.9990e-04,  8.1566e-06]]], grad_fn=<ViewBackward0>),
 tensor([[-0.0743,  0.6403, -0.4663],
         [ 0.0646, -0.5416, -0.9011],
         [ 0.1294,  0.7737, -0.2321],
         [-0.1226,  0.7680, -0.4474]], grad_fn=<AddmmBackward0>))

test topp routing:NEED to add MoEBlock to the __init__

In [6]:

# Create a minimal config
config = mdl.NeoBERTConfig(
    vocab_size=100,            # must be > max index in dummy_input
    hidden_size=2,             # divisible by num_attention_heads
    num_attention_heads=2,     # divides hidden_size evenly
    num_hidden_layers=2,       # small number of layers
    intermediate_size=16,      # FFN dim
    max_length=16,             # match seq_len
    pad_token_id=0,
    routing_strategy="top_p",
    num_experts_per_tok_inference=1,
    min_expert_cumprob_per_token = 0.6,

                                                                                # define padding token index
    rope=False,                # simpler
    flash_attention=False      # simpler
)

# Instantiate mode

# Generate dummy input (ensure all token indices < vocab_si14ze)
batch_size = 1
seq_len = 4

test_input = torch.randn(batch_size,seq_len,config.hidden_size)
test_input_shape = test_input.shape
print("test_input: ", test_input)

MoeBlock = mdl.MoEBlock(config)
MoeBlock.eval()

MoeBlock(hidden_states = test_input, output_expert_usage_loss = False)


test_input:  tensor([[[ 0.3110,  0.9468],
         [ 0.3964, -0.0248],
         [-0.8394,  0.3342],
         [-0.5307,  0.0278]]])
hidden_states tensor([[ 0.3110,  0.9468],
        [ 0.3964, -0.0248],
        [-0.8394,  0.3342],
        [-0.5307,  0.0278]])
routing_weights tensor([[0.4735, 0.3135, 0.2131],
        [0.3557, 0.2626, 0.3817],
        [0.4059, 0.1468, 0.4473],
        [0.3604, 0.1580, 0.4816]], grad_fn=<SoftmaxBackward0>)
tensor([[0],
        [1],
        [2]])
expert_idx tensor([0])
token_idx tensor([0, 1, 2, 3])
current_states tensor([[ 0.3110,  0.9468],
        [ 0.3964, -0.0248],
        [-0.8394,  0.3342],
        [-0.5307,  0.0278]])
expert mask shape torch.Size([3, 4])
rtouting weights shape torch.Size([4, 3])
selected_sum tensor([0.7869, 0.7374, 0.8532, 0.8420], grad_fn=<SumBackward1>)
selected_weights tensor([0.6017, 0.4823, 0.4758, 0.4280], grad_fn=<DivBackward0>)
expert_output tensor([[ 0.3110,  0.9468],
        [ 0.3964, -0.0248],
        [-0.8394,  0.3342],
  

RuntimeError: output with shape [1] doesn't match the broadcast shape [4]

test model with losses

In [2]:
import importlib
import neobert.model as mdl
importlib.reload(mdl)
import neobert.pretraining.losses as lss



c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\NeoBERTenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Create a minimal config
config_basis = {
  "vocab_size": 100,
  "hidden_size": 8,
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "intermediate_size": 16,
  "max_length": 16,
  "pad_token_id": 0,
  "rope": False,
  "flash_attention": False
}

# Generate dummy input (ensure all token indices < vocab_size)
batch_size = 2
seq_len = 16

# Token indices from 1 to vocab_size-1 (reserve 0 for pad_token)
dummy_input = torch.randint(1, config.vocab_size, (batch_size, seq_len))

# Add some padding tokens at the end (optional)
dummy_input[:, -2:] = config.pad_token_id  # last 2 tokens are padding

# Generate additive pad mask: 0.0 for valid tokens, -inf for padding
pad_mask = (dummy_input != config.pad_token_id).float()
pad_mask = torch.where(pad_mask == 1, 0.0, float('-inf'))


#mop
print("MOP forward  pass")
config_mop = {
    'routing_strategy': "top_k",
    'num_experts_per_tok_training': 2,
    'cost_based_loss_alpha': 1e-5,
    'cost_based_loss_epsilon': 1e-2,
    'disable_task_performance_scaling': False
}

config = mdl.NeoBERTConfig(**config_basis,**config_mop)     # simple
model = mdl.NeoBERTLMHead(config)

# Run forward pass
model.training()
model_output = model(dummy_input, pad_mask=pad_mask, output_expert_usage_loss=True)

#hetero moe
print("hetero moe forward pass")
config_mop = {
    'routing_strategy': "top_k",
    'num_experts_per_tok_training': 2,
    'cost_based_loss_alpha': 1e-5,
    'cost_based_loss_epsilon': 1e-2,
    'disable_task_performance_scaling': False
}


load_balancing_loss = lls.load_balancing_loss(gate_logits = model_output['router_logits'], num_experts = 9)
penalty_loss = lls.hmoe_penalty_loss_fn(gate_logits = model_output['router_logits'], num_experts = 9)
entropy_loss = lls.hmoe_entropy_loss_fn()


# if cfg.model.type == "homo_moe":
#                         model_output = model(batch["input_ids"], batch.get("attention_mask", None), output_expert_usage_loss=False, output_router_logits=True)
#                         train_loss = homo_moe_loss_fn(model_output['logits'], model_output['router_logits'], cfg, batch)
#                     if cfg.model.type == "hetero_moe":
#                         model_output = model(batch["input_ids"], batch.get("attention_mask", None), output_expert_usage_loss=False, output_router_logits=True)
#                         train_loss = hetero_moe_loss_fn(model_output['logits'], model_output['router_logits'], cfg, batch)
#                     if cfg.model.type == "mop":
#                         model_output = model(batch["input_ids"], batch.get("attention_mask", None), output_expert_usage_loss=True, output_router_logits=False)
#                         train_loss = mop_loss_fn(model_output['logits'], model_output['expert_usage_loss'], cfg, batch) 

#                     logits = model_output['logits']






IndentationError: unindent does not match any outer indentation level (<string>, line 4)

In [38]:
import neobert.model as mdl  # or your actual import if different

# Create a minimal config
config = mdl.NeoBERTConfig(        # must be > max index in dummy_input
    hidden_size=768,             # divisible by num_attention_heads
    num_attention_heads=12,     # divides hidden_size evenly
    num_hidden_layers=12,       # small number of layers
    intermediate_size=3072,      # FFN dim
    max_length=128,             # match seq_len
    pad_token_id=0,            # define padding token index
    rope=False, 
    vocab_size = 32768,
    expert_sizes = '0,504, 616, 728, 840, 952, 1064, 1176',              # simpler
    flash_attention=False      # simpler
)

# Instantiate model
model = mdl.NeoBERTLMHead(config)
# sd = model.state_dict()
# for k,v in sd.items(): 
#     print(k,v.shape)
    
 # plot gradients of the gates and of the experts

num_gate_params = 0.0
num_expert_params = 0.0
num_embedding_params = 0.0
num_attention_params = 0.0

for name, param in model.named_parameters():
        if "gate" in name:
            num_gate_params += param.numel()
        if "experts" in name:
            num_expert_params += param.numel()
        if "model.encoder" in name or "decoder" in name or "model.positional_embedding" in name:
            num_embedding_params += param.numel()
        if "qkv" in name or "wo" in name:
            num_attention_params += param.numel()

total_num_params = num_gate_params + num_expert_params + num_embedding_params + num_attention_params
gate_prop= num_gate_params / total_num_params
attention_prop = num_attention_params / total_num_params
embedding_prop = num_embedding_params / total_num_params
expert_prop = num_expert_params / total_num_params

print(f"gate_prop:{gate_prop*100:.2f}%")
print(f"attention_prop:{attention_prop*100:.2f}%")
print(f"embedding_prop:{embedding_prop*100:.2f}%")
print(f"expert_prop:{expert_prop*100:.2f}%")

# print("gate_grad_norm:", gate_grad_norm ** 0.5 / num_gate_params)
# print("expert_grad_norm:", expert_grad_norm ** 0.5 / num_expert_params)
# print("embedding_grad_norm:", embedding_grad_norm ** 0.5 / num_embedding_params)


gate_prop:0.04%
attention_prop:15.16%
embedding_prop:27.02%
expert_prop:57.79%


In [54]:
print(32768/2
      )

16384.0


In [42]:
import numpy as np
np.array([0,504, 616, 728, 840, 952, 1064, 1176])//2


array([  0, 252, 308, 364, 420, 476, 532, 588], dtype=int32)

In [66]:
config = mdl.NeoBERTConfig(        # must be > max index in dummy_input
    hidden_size=120,             # divisible by num_attention_heads
    num_attention_heads=12,     # divides hidden_size evenly
    num_hidden_layers=12,       # small number of layers
    intermediate_size=3072,      # FFN dim
    max_length=128,             # match seq_len
    pad_token_id=0,            # define padding token index
    rope=False, 
    vocab_size = 32768,
    expert_sizes = '0, 252, 308, 364, 420, 476, 532, 588',              # simpler
    flash_attention=False      # simpler
)

model = mdl.NeoBERTLMHead(config)
# sd = model.state_dict()
# for k,v in sd.items(): 
#     print(k,v.shape)
    
 # plot gradients of the gates and of the experts

num_gate_params = 0.0
num_expert_params = 0.0
num_embedding_params = 0.0
num_attention_params = 0.0

for name, param in model.named_parameters():
        if "gate" in name:
            num_gate_params += param.numel()
        if "experts" in name:
            num_expert_params += param.numel()
        if "model.encoder" in name or "decoder" in name or "model.positional_embedding" in name:
            num_embedding_params += param.numel()
        if "qkv" in name or "wo" in name:
            num_attention_params += param.numel()

total_num_params = num_gate_params + num_expert_params + num_embedding_params + num_attention_params
gate_prop= num_gate_params / total_num_params
attention_prop = num_attention_params / total_num_params
embedding_prop = num_embedding_params / total_num_params
expert_prop = num_expert_params / total_num_params
print(f"total_num_params:{total_num_params}")

print(f"gate_prop:{gate_prop*100:.2f}%")
print(f"attention_prop:{attention_prop*100:.2f}%")
print(f"embedding_prop:{embedding_prop*100:.2f}%")
print(f"expert_prop:{expert_prop*100:.2f}%")


total_num_params:17013368.0
gate_prop:0.07%
attention_prop:4.06%
embedding_prop:46.51%
expert_prop:49.36%


In [74]:
import numpy as np
#expert_loss
expert_sizes = np.array([0, 252, 308, 364, 420, 476, 532, 588])
somme = sum(expert_sizes**2)
num_layers = 12
num_experts = len(expert_sizes)
alpha = 1e-8
vocab_size = 30522


#expert_loss
mlm_loss = -np.log(1/vocab_size)
initial_expert_loss = (num_layers*somme*alpha)/(num_experts*mlm_loss)
print("initial_expert_loss:", initial_expert_loss)

#penalty_loss
expert_sizes = np.array([196, 252, 308, 364, 420, 476, 532, 588])
coeff_penalty =1.5e-6 #1e-6
top_p = 0.4
# 0.125*4 = 0.5
# 0.125*3 = 0.375
# donc a priori il faut 4 experts activés au début. 
num_experts = len(expert_sizes)
mean_expert_size = sum(expert_sizes)/num_experts

prop_3 = 3.75/4
prop_4 = 1-3.75/4

initial_penalty_loss = coeff_penalty *(prop_3*3+prop_4*4)*mean_expert_size
print("initial_penalty_loss:", initial_penalty_loss)

initial_expert_loss: 0.0019212405540550573
initial_penalty_loss: 0.0018007499999999998


In [5]:
print(somme)

1322608


In [11]:
(252**2)*8

508032

In [10]:
(588**2)*8

2765952

In [6]:

expert_sizes = np.array([0,504, 616, 728, 840, 952, 1064, 1176])
somme = sum(expert_sizes**2)
num_layers = 12
num_experts = len(expert_sizes)
alpha = 1e-8
vocab_size = 30522
mlm_loss = -np.log(1/vocab_size)
print((num_layers*somme*alpha)/(num_experts*mlm_loss))

0.007684962216220229


661304.0


In [81]:
from transformers import BertModel, BertForMaskedLM
model = BertModel.from_pretrained("bert-base-uncased")
model_LM = BertForMaskedLM.from_pretrained("bert-base-uncased")
for s, k in model.named_parameters():
    print(s, k.size())
print("")
print("")
for s, k in model_LM.named_parameters():
    print(s, k.size())


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


embeddings.word_embeddings.weight torch.Size([30522, 768])
embeddings.position_embeddings.weight torch.Size([512, 768])
embeddings.token_type_embeddings.weight torch.Size([2, 768])
embeddings.LayerNorm.weight torch.Size([768])
embeddings.LayerNorm.bias torch.Size([768])
encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
encoder.layer.0.attention.self.query.bias torch.Size([768])
encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
encoder.layer.0.attention.self.key.bias torch.Size([768])
encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
encoder.layer.0.attention.self.value.bias torch.Size([768])
encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
encoder.layer.0.attention.output.dense.bias torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
encoder.layer.0.intermediate.dense.weight torch.Size([3072, 768])
encoder.layer.0.inter

In [ ]:
import neobert.model as mdl  # or your actual import if different

# Create a minimal config
config = mdl.NeoBERTConfig(        # must be > max index in dummy_input
    hidden_size=16,             # divisible by num_attention_heads
    num_attention_heads=2,     # divides hidden_size evenly
    num_hidden_layers=4,       # small number of layers
    intermediate_size=3072,      # FFN dim
    max_length=128,             # match seq_len
    pad_token_id=0,            # define padding token index
    rope=False, 
    vocab_size = 30522,
    expert_sizes = '0,504, 616, 728, 840, 952, 1064, 1176',              # simpler
    flash_attention=False      # simpler
)

# Generate dummy input (ensure all token indices < vocab_size)
batch_size = 2
seq_len = 128
num_experts = 8

# Token indices from 1 to vocab_size-1 (reserve 0 for pad_token)
dummy_input = torch.randint(1, config.vocab_size, (batch_size, seq_len))

# Add some padding tokens at the end (optional)
dummy_input[:, -2:] = config.pad_token_id  # last 2 tokens are padding

# print("dummy input:", dummy_input)
print("dummy input shape:", dummy_input.shape)

# Generate additive pad mask: 0.0 for valid tokens, -inf for padding
pad_mask = (dummy_input != config.pad_token_id).float()
pad_mask = torch.where(pad_mask == 1, 0.0, float('-inf'))
#pad_mask = None

#print(pad_mask.view(256,1))



# Instantiate model
# sd = model.state_dict()
# for k,v in sd.items(): 
#     print(k,v.shape)


In [159]:
a = torch.randn(2,2)
print(a)
b = torch.tensor([True, False])

print(a[b])

tensor([[-1.5254,  1.1156],
        [ 1.0100,  0.7761]])
tensor([[-1.5254,  1.1156]])


In [173]:



from transformers import BertModel
import torch
import torch.nn as nn


class RouterPredictionHead(nn.Module):
        def __init__(self, num_experts, hidden_size, num_hidden_layers):
            super().__init__()
            self.mlp_block = nn.ModuleList(nn.Sequential(nn.Linear(hidden_size, hidden_size),
                            nn.Linear(hidden_size, num_experts)) for _ in range(num_hidden_layers))

        def forward(self, x): #batch_dim, seq_length, hidden_size
            outputs = tuple(mlp(x) for mlp in self.mlp_block)## tuple of n_layers tensors of size batch_dim, seq_length, n_experts
            print("RouterPredictionHead outputs shape:", outputs[0].shape)
            outputs = tuple(layer_output.view(-1, layer_output.size(-1)) for layer_output in outputs)
            print("RouterPredictionHead outputs shape:", outputs[0].shape)
            return  outputs  # tuple of n_layers tensors of size batch_dim*seq_length, n_experts

class BERTPredictor(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        for param in self.bert.parameters():
            param.requires_grad = False
        #pretrained bert params
        hidden_size = self.bert.config.hidden_size
        #neobert layer params
        num_hidden_layers = cfg.num_hidden_layers
        num_experts = len(cfg.expert_sizes.split(","))

        self.router_head = RouterPredictionHead(num_experts, hidden_size, num_hidden_layers)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        last_hidden_state = outputs.last_hidden_state
        logits = self.router_head(last_hidden_state)
        return logits
    

def get_expert_mask(gate_logits):
    if isinstance(gate_logits, tuple):
        compute_device = gate_logits[0].device
        stacked_gate_logits = torch.stack([layer_gate.to(compute_device) for layer_gate in gate_logits], dim=0)#n_layers, batch_size*seq_length, n_experts
        _,_, num_experts = stacked_gate_logits.shape
    routing_weights = torch.nn.functional.softmax(stacked_gate_logits, dim=-1)

    top_k = 2
    _, selected_experts = torch.topk(routing_weights, top_k, dim=-1)
    target_expert_mask = torch.nn.functional.one_hot(selected_experts, num_experts).sum(2)#n_layers, batch_size*seq_length, n_experts

    return target_expert_mask #n_layers, batch_size*seq_length, n_experts


    #load  pretrained config

# get pretrained BERT predictor
BERTpredictor = BERTPredictor(config)

# get pretrained neobert model
neobert_model = mdl.NeoBERTLMHead(config)
#freeze params
neobert_model.eval()
for params in neobert_model.parameters():
    params.requires_grad = False
#get dataset
# define loss function
loss_fn = nn.BCEWithLogitsLoss()
#define optimizer

#finetuning on layer prediction



#compute the target routing paths
neobert_model_output = neobert_model(dummy_input, output_expert_usage_loss=False, output_router_logits=True)
target_gate_logits = neobert_model_output['router_logits'] #tuple  of n_layers of batch_size*seq_length,  n_experts
print("shape output neobert:", target_gate_logits[0].shape)
target_expert_mask = get_expert_mask(target_gate_logits) # n_layers*batch_size*n_seq, n_experts
print("target_expert_mask shpe:", target_expert_mask.shape)

predicted_routed_logits = BERTpredictor(dummy_input)
print("predicted_routed_logits shape:", predicted_routed_logits[0].shape)
predicted_routed_logits = torch.stack(predicted_routed_logits, dim=0) #n_layers, batch_size*seq_length, n_experts
print("predicted_routed_logits shape:", predicted_routed_logits.shape)


#ignore padded tokens
if pad_mask is not None:
    pad_mask = pad_mask.view(-1, 1)
    print(pad_mask)
    pad_mask = (pad_mask != float("-inf")).squeeze(-1)
    print("pad mask shape", pad_mask.shape)
    print(pad_mask)
    print("predicted routed logits shape", predicted_routed_logits.shape)
    predicted_routed_logits = predicted_routed_logits[:,pad_mask,:]
    print("target expert mask shape", target_expert_mask.shape)
    target_expert_mask = target_expert_mask[:,pad_mask,:]
    print("target expert mask shape", target_expert_mask.shape)

#reshape 
predicted_routed_logits = predicted_routed_logits.view(-1, predicted_routed_logits.size(-1)) #n_layers*batch_size*seq_length, n_experts
target_expert_mask = target_expert_mask.view(-1, target_expert_mask.size(-1)) #n_layers*batch_size*seq_length, n_experts




loss = loss_fn(predicted_routed_logits, target_expert_mask*1.0)
loss.backward()

# Update model parameters

    #testing
BERTpredictor.eval()
with torch.no_grad():
    total_acc =[]
    #target
    model_output = neobert_model(dummy_input, output_expert_usage_loss=False, output_router_logits=True)
    target_gate_logits = model_output['router_logits'] #tuple  of n_layers of batch_size*seq_length,  n_experts
    target_expert_mask = get_expert_mask(target_gate_logits) # n_layers*batch_size*n_seq, n_experts

    #prediction
    predicted_routed_logits = BERTpredictor(dummy_input)
    predicted_routed_logits = torch.stack(predicted_routed_logits, dim=0) #n_layers, batch_size*seq_length, n_experts
    predicted_expert_mask = get_expert_mask(predicted_routed_logits)

    if pad_mask is not None:
        pad_mask = pad_mask.view(-1, 1)
        print(pad_mask)
        pad_mask = (pad_mask != float("-inf")).squeeze(-1)
        print("pad mask shape", pad_mask.shape)
        print(pad_mask)
        print("predicted routed logits shape", predicted_routed_logits.shape)
        predicted_routed_logits = predicted_routed_logits[:,pad_mask,:]
        print("target expert mask shape", target_expert_mask.shape)
        target_expert_mask = target_expert_mask[:,pad_mask,:]
        print("target expert mask shape", target_expert_mask.shape)

    #compute accuracy between target mask and predicted mask
    layer_accuracy =torch.sum(target_expert_mask*predicted_expert_mask, dim = 1)/torch.sum(target_expert_mask, dim = 1)
    mean_accuracy = torch.mean(layer_accuracy)
    total_acc.append(mean_accuracy)
    mean_accuracy = torch.stack(total_acc).mean()
    print(mean_accuracy.item())


shape output neobert: torch.Size([256, 8])
target_expert_mask shpe: torch.Size([4, 256, 8])
RouterPredictionHead outputs shape: torch.Size([2, 128, 8])
RouterPredictionHead outputs shape: torch.Size([256, 8])
predicted_routed_logits shape: torch.Size([256, 8])
predicted_routed_logits shape: torch.Size([4, 256, 8])
tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
    

UnboundLocalError: cannot access local variable 'stacked_gate_logits' where it is not associated with a value

In [ ]:
from omegaconf import DictConfig, OmegaConf

# Create a DictConfig with all keys used in mop_loss_fn
cfg = DictConfig({
    "tokenizer": {
        "vocab_size": 32768  # Example vocab size for BERT-like tokenizer
    },
    "model": {
        "loss": {
            "cost_based_loss_alpha": 1e-5,
            "cost_based_loss_epsilon": 1e-2,
            "disable_task_performance_scaling": False
        }
    }
})

# Pretty-print for verification
print(OmegaConf.to_yaml(cfg))


tokenizer:
  vocab_size: 30522
model:
  loss:
    cost_based_loss_alpha: 0.1
    cost_based_loss_epsilon: 1.0e-06
    disable_task_performance_scaling: false



In [11]:
a = torch.tensor(3)
print(torch.tensor([2,3,a]))

tensor([2, 3, 3])


Test load balancing loss

In [44]:
from typing import Union, Optional,Dict,Any,Tuple

def load_balancing_loss_fn(
    gate_logits: Union[torch.Tensor, tuple[torch.Tensor], None],
    num_experts: Optional[int] = None,
    top_k=2,
    attention_mask: Optional[torch.Tensor] = None,
) -> Union[torch.Tensor, int]:
    r"""
    Computes auxiliary load balancing loss as in Switch Transformer - implemented in Pytorch.

    See Switch Transformer (https://huggingface.co/papers/2101.03961) for more details. This function implements the loss
    function presented in equations (4) - (6) of the paper. It aims at penalizing cases where the routing between
    experts is too unbalanced.

    Args:
        gate_logits:
            Logits from the `gate`, should be a tuple of model.config.num_hidden_layers tensors of
            shape [batch_size X sequence_length, num_experts].
        num_experts:
            Number of experts
        top_k:
            The number of experts to route per-token, can be also interpreted as the `top-k` routing
            parameter.
        attention_mask (`torch.Tensor`, *optional*):
            The attention_mask used in forward function
            shape [batch_size X sequence_length] if not None.

    Returns:
        The auxiliary loss.
    """
    if gate_logits is None or not isinstance(gate_logits, tuple):
        return 0

    if isinstance(gate_logits, tuple):
        compute_device = gate_logits[0].device
        concatenated_gate_logits = torch.cat([layer_gate.to(compute_device) for layer_gate in gate_logits], dim=0)

    routing_weights = torch.nn.functional.softmax(concatenated_gate_logits, dim=-1)

    _, selected_experts = torch.topk(routing_weights, top_k, dim=-1)

    expert_mask = torch.nn.functional.one_hot(selected_experts, num_experts)

    if attention_mask is None:
        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.mean(expert_mask.float(), dim=0)
        print("tokens per expert:", tokens_per_expert)

        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.mean(routing_weights, dim=0)
        print("router prob per expert:", router_prob_per_expert)
    else:
        attention_mask= torch.where(attention_mask == float(0.0), 1, float(0.0)) #original mask is additive-->1,0
        tokens_per_expert = torch.mean(expert_mask.float(), dim=0)
        print("tokens per expert:", tokens_per_expert)

        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.mean(routing_weights, dim=0)
        print("router prob per expert:", router_prob_per_expert)

        prod= tokens_per_expert * router_prob_per_expert.unsqueeze(0)
        print(prod)
        print(torch.sum(prod))
        
        batch_size, sequence_length = attention_mask.shape
        num_hidden_layers = concatenated_gate_logits.shape[0] // (batch_size * sequence_length)

        # Compute the mask that masks all padding tokens as 0 with the same shape of expert_mask
        expert_attention_mask = (
            attention_mask[None, :, :, None, None]
            .expand((num_hidden_layers, batch_size, sequence_length, top_k, num_experts))
            .reshape(-1, top_k, num_experts)
            .to(compute_device)
        )

        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.sum(expert_mask.float() * expert_attention_mask, dim=0) / torch.sum(
            expert_attention_mask, dim=0
        )

        # Compute the mask that masks all padding tokens as 0 with the same shape of tokens_per_expert
        router_per_expert_attention_mask = (
            attention_mask[None, :, :, None]
            .expand((num_hidden_layers, batch_size, sequence_length, num_experts))
            .reshape(-1, num_experts)
            .to(compute_device)
        )

        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.sum(routing_weights * router_per_expert_attention_mask, dim=0) / torch.sum(
            router_per_expert_attention_mask, dim=0
        )
    print("after padding:")
    print("tokens per expert:", tokens_per_expert)
    print("router prob per expert:", router_prob_per_expert)
    prod= tokens_per_expert * router_prob_per_expert.unsqueeze(0)
    print(prod)
    print(torch.sum(prod))

    overall_loss = torch.sum(tokens_per_expert * router_prob_per_expert.unsqueeze(0))
    return overall_loss * num_experts

In [99]:
def hmoe_penalty_loss_fn(
    gate_logits: Union[torch.Tensor, tuple[torch.Tensor], None],
    cfg: Dict[str, Any],
    num_experts: Optional[int] = None,
    attention_mask: Optional[torch.Tensor] = None,
) -> Union[torch.Tensor, int]:
    r"""
    Computes auxiliary load balancing loss as in Switch Transformer - implemented in Pytorch.

    See Switch Transformer (https://huggingface.co/papers/2101.03961) for more details. This function implements the loss
    function presented in equations (4) - (6) of the paper. It aims at penalizing cases where the routing between
    experts is too unbalanced.

    Args:
        gate_logits:
            Logits from the `gate`, should be a tuple of model.config.num_hidden_layers tensors of
            shape [batch_size X sequence_length, num_experts].
        num_experts:
            Number of experts
        cfg:
            The configuration object
        attention_mask (`torch.Tensor`, *optional*):
            The attention_mask used in forward function
            shape [batch_size X sequence_length] if not None.

    Returns:
        The auxiliary loss.
    """
    if gate_logits is None or not isinstance(gate_logits, tuple):
        return 0

    if isinstance(gate_logits, tuple):
        compute_device = gate_logits[0].device
        concatenated_gate_logits = torch.cat([layer_gate.to(compute_device) for layer_gate in gate_logits], dim=0)

    routing_weights = torch.nn.functional.softmax(concatenated_gate_logits, dim=-1)
    expert_sizes = torch.tensor([int(size) for size in cfg.expert_sizes.split(",")], device=compute_device)

    if cfg.routing_strategy == "top_k":
        top_k = 2 #if we wanted  to use top_k for heterogeneous moe
        _, selected_experts = torch.topk(routing_weights, top_k, dim=-1)

        expert_mask = torch.nn.functional.one_hot(selected_experts, num_experts)# n_tokens, top_k, num_experts
        expert_mask = torch.sum(expert_mask, dim=1) * expert_sizes  # n_tokens, num_experts
        mean_num_activated_experts = top_k
    elif cfg.routing_strategy == "top_p":
        top_p = cfg.min_expert_cumprob_per_token
        sorted_weights, sorted_indices = torch.sort(routing_weights, dim=-1, descending=False)

        cum_probs = sorted_weights.cumsum(dim=-1)
        mask = cum_probs > 1 - top_p

        unsorted_mask = torch.zeros_like(mask, dtype=torch.bool)
        #expert_mask = unsorted_mask.scatter(dim=-1, index=sorted_indices, src=mask)*expert_sizes
        expert_mask = unsorted_mask.scatter(dim=-1, index=sorted_indices, src=mask).float()#n_layers*batch_size*seq_len, num_experts
        mean_num_activated_experts = torch.mean(torch.sum(expert_mask.float(), dim=1), dim=0)
        expert_mask *= expert_sizes  # n_tokens, num_experts

    if attention_mask is None:
        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.mean(expert_mask.float(), dim=0)

        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.mean(routing_weights, dim=0)
    else:
        # tokens_per_expert = torch.mean(expert_mask.float(), dim=0)

        # # Compute the average probability of routing to these experts
        # router_prob_per_expert = torch.mean(routing_weights, dim=0)

        attention_mask= torch.where(attention_mask == float(0.0), 1, float(0.0)) #original mask is additive-->1,0

        batch_size, sequence_length = attention_mask.shape
        num_hidden_layers = concatenated_gate_logits.shape[0] // (batch_size * sequence_length)
        print(num_hidden_layers)

        # Compute the mask that masks all padding tokens as 0 with the same shape of expert_mask
        expert_attention_mask = (
            attention_mask[None, :, :,None]
            .expand((num_hidden_layers, batch_size, sequence_length,num_experts))
            .reshape(-1, num_experts)
            .to(compute_device)
        )

        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.sum(expert_mask.float() * expert_attention_mask, dim=0) / torch.sum(
            expert_attention_mask, dim=0
        )
        print("tokens per expert:", tokens_per_expert)
        print("tokens per expert shape:", tokens_per_expert.shape)

        # Compute the mask that masks all padding tokens as 0 with the same shape of tokens_per_expert
        router_per_expert_attention_mask = (
            attention_mask[None, :, :, None]
            .expand((num_hidden_layers, batch_size, sequence_length, num_experts))
            .reshape(-1, num_experts)
            .to(compute_device)
        )
        
        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.sum(routing_weights * router_per_expert_attention_mask, dim=0) / torch.sum(
            router_per_expert_attention_mask, dim=0
        )

    overall_loss = torch.sum(tokens_per_expert * router_prob_per_expert.unsqueeze(0))
    return overall_loss * num_experts, mean_num_activated_experts

In [102]:


# Create a minimal config
config = mdl.NeoBERTConfig(        # must be > max index in dummy_input
    hidden_size=16,             # divisible by num_attention_heads
    num_attention_heads=2,     # divides hidden_size evenly
    num_hidden_layers=4,       # small number of layers
    intermediate_size=3072,      # FFN dim
    max_length=128,             # match seq_len
    pad_token_id=0,
    routing_strategy = "top_p",            # define padding token index
    rope=False, 
    vocab_size = 30522,
    expert_sizes = '0,504, 616, 728, 840, 952, 1064, 1176',              # simpler
    flash_attention=False      # simpler
)

# Generate dummy input (ensure all token indices < vocab_size)
batch_size = 2
seq_len = 128
num_experts = 8

# Token indices from 1 to vocab_size-1 (reserve 0 for pad_token)
dummy_input = torch.randint(1, config.vocab_size, (batch_size, seq_len))

# Add some padding tokens at the end (optional)
dummy_input[:, -2:] = config.pad_token_id  # last 2 tokens are padding

# print("dummy input:", dummy_input)
print("dummy input shape:", dummy_input.shape)

# Generate additive pad mask: 0.0 for valid tokens, -inf for padding
pad_mask = (dummy_input != config.pad_token_id).float()
pad_mask = torch.where(pad_mask == 1, 0.0, float('-inf'))
#pad_mask = None

dummy input shape: torch.Size([2, 128])


In [103]:
neobert_model = mdl.NeoBERTLMHead(config)
neobert_model_output = neobert_model(dummy_input, output_expert_usage_loss=False, output_router_logits=True)
gate_logits = neobert_model_output["router_logits"]
penalty_loss, mean_num_activated_experts = hmoe_penalty_loss_fn(gate_logits,config, num_experts=num_experts, attention_mask=pad_mask)
print(penalty_loss)
print(mean_num_activated_experts)


4
tokens per expert: tensor([  0.0000, 242.5000, 306.1667, 371.9445, 438.3333, 444.8333, 526.7222,
        595.0000])
tokens per expert shape: torch.Size([8])
router prob per expert shape: torch.Size([8])
tensor(365.7164, grad_fn=<SumBackward0>)
tensor(2925.7314, grad_fn=<MulBackward0>)
tensor(3.9766)


In [14]:
neobert_model = mdl.NeoBERTLMHead(config)

neobert_model_output = neobert_model(dummy_input, output_expert_usage_loss=False, output_router_logits=True)
gate_logits = neobert_model_output["router_logits"]
load_balancing_loss = load_balancing_loss_fn(gate_logits, num_experts=num_experts, attention_mask=pad_mask)
print(load_balancing_loss)


NameError: name 'dummy_input' is not defined

Test the difficulty measure

In [15]:
config = mdl.NeoBERTConfig(        # must be > max index in dummy_input
    hidden_size=16,             # divisible by num_attention_heads
    num_attention_heads=2,     # divides hidden_size evenly
    num_hidden_layers=4,       # small number of layers
    intermediate_size=3072,      # FFN dim
    max_length=128,             # match seq_len
    pad_token_id=0,            # define padding token index
    rope=False, 
    vocab_size = 30522,
    expert_sizes = '0,504, 616, 728, 840, 952, 1064, 1176',              # simpler
    flash_attention=False      # simpler
)

# Generate dummy input (ensure all token indices < vocab_size)
batch_size = 2
seq_len = 128
num_experts = 8

In [48]:
experts = torch.tensor([0,504, 616, 728, 840, 952, 1064, 1176])
somme = torch.sum(experts)
a  = (experts/somme)**2
print(a.mean()*4)
#expected initial value


tensor(0.0765)


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
difmeasure_model_name = "textattack/bert-base-uncased-STS-B"
    #we check that
tokenizer = AutoTokenizer.from_pretrained(difmeasure_model_name)
difmeasure_model = AutoModelForSequenceClassification.from_pretrained(difmeasure_model_name)
difmeasure_model.eval()  # evaluation mode

c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\NeoBERTenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gruau\.cache\huggingface\hub\models--textattack--bert-base-uncased-STS-B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [17]:
from pathlib import Path
from datasets import load_dataset, load_from_disk
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)#not sure why
from typing import Optional

def prepare_and_cache_stsb(tokenizer, cache_dir: Optional[str] = None):
    """
    Download, tokenize, and cache the GLUE STSB dataset with only necessary columns.
    Only tokenized columns and label are kept.
    """
    stsb = load_dataset("glue", "stsb")  # DatasetDict with splits

    def tokenize_fn(batch):
        # Tokenize both sentences
        tokens = tokenizer(
            batch["sentence1"],
            batch["sentence2"],
            truncation=True,
            max_length=128,
            padding="max_length",
        )
        # Only add label once
        tokens["label"] = batch["label"]
        return tokens

    # Remove sentence1 and sentence2 from the output
    columns_to_remove = ["sentence1", "sentence2"]
    # Also remove any other columns except label and idx if you want
    for split in stsb.keys():
        # Find all columns except label and idx
        all_columns = stsb[split].column_names
        remove_cols = [col for col in all_columns if col not in ["label", "idx"]]
        stsb[split] = stsb[split].map(tokenize_fn, batched=True, remove_columns=remove_cols)

    # Choose cache dir
    if cache_dir is None:
        base_dir = Path("/data") if Path("/data").exists() else Path.home()
        cache_dir = base_dir / ".pathways_cache" / "glue_stsb_tokenized"
    else:
        cache_dir = Path(cache_dir)
    cache_dir.mkdir(parents=True, exist_ok=True)
    # Save each split
    for split in stsb.keys():
        stsb[split].save_to_disk(cache_dir / split)
    logger.info(f"Tokenized STSB saved to {cache_dir}")

def load_cached_stsb(tokenizer, cache_dir: Optional[str] = None) -> dict:
    """
    Load cached tokenized STSB splits from disk.
    If not all splits are found, prepare and cache them first.
    Returns a dict: {"train": Dataset, "validation": Dataset, "test": Dataset}
    """
    if cache_dir is None:
        base_dir = Path("/data") if Path("/data").exists() else Path.home()
        cache_dir = base_dir / ".pathways_cache" / "glue_stsb_tokenized"
    else:
        cache_dir = Path(cache_dir)
    splits = ["train", "validation", "test"]
    datasets = {}
    missing = []
    for split in splits:
        split_path = cache_dir / split
        if split_path.exists():
            datasets[split] = load_from_disk(split_path)
        else:
            missing.append(split)
    if missing:
        logger.info(f"Missing splits {missing} in cache. Preparing and caching STSB dataset.")
        prepare_and_cache_stsb(tokenizer, cache_dir=cache_dir)
        for split in missing:
            split_path = cache_dir / split
            datasets[split] = load_from_disk(split_path)
    return datasets

In [49]:
cached_dir = Path(".cache")
tokenizer_text_attack = AutoTokenizer.from_pretrained(difmeasure_model_name)
splits = load_cached_stsb(tokenizer_text_attack, cache_dir=cached_dir)

stsb_train_dataset = splits["train"]
stsb_validation_dataset = splits["validation"]

In [50]:
print(stsb_train_dataset['idx'])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [51]:
from torch.utils.data import DataLoader
class STSBDataCollator:
    """
    Collates batches for STSB regression task from already-tokenized datasets.
    Converts attention_mask to additive mask for NeoBERT.
    """
    def __init__(self, dtype_pad_mask):
        self.dtype_pad_mask = dtype_pad_mask

    def __call__(self, batch):
        # batch: list of dicts with tokenized fields
        input_ids = torch.stack([torch.tensor(ex["input_ids"]) for ex in batch])
        # Standard attention mask: 1 for tokens, 0 for padding
        attention_mask = torch.stack([torch.tensor(ex["attention_mask"]) for ex in batch])
        # Convert to additive mask: 0 for tokens, -inf for padding
        additive_mask = torch.where(attention_mask == 1, 0.0, float('-inf')).to(dtype = self.dtype_pad_mask)
        labels = torch.tensor([float(ex["label"]) for ex in batch], dtype=torch.float32)
        collated = {
            "input_ids": input_ids,
            "attention_mask": additive_mask,
            "labels": labels
        }
        # If token_type_ids exist (for BERT), add them
        if "token_type_ids" in batch[0]:
            collated["token_type_ids"] = torch.stack([torch.tensor(ex["token_type_ids"]) for ex in batch])
        return collated
    
stsb_collator = STSBDataCollator(torch.float32)

val_dataloader = DataLoader(
    stsb_validation_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=stsb_collator
)

train_dataloader = DataLoader(
    stsb_train_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=stsb_collator
)

In [36]:
i = 0 
for batch in train_dataloader:
    print(batch.keys())
    i += 1
    if i >= 2:  # Limit to 5 batches
        break

dict_keys(['input_ids', 'attention_mask', 'labels', 'token_type_ids'])
dict_keys(['input_ids', 'attention_mask', 'labels', 'token_type_ids'])


In [57]:
import pandas as pd
results = []
for batch in val_dataloader:
    # Get batch input_ids, attention_mask, labels, idx if available
    input_ids = batch["input_ids"]
    
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    idxs = batch.get("idx", None)
    # If you need sentence1/sentence2, you must keep them in the dataset and collator

    attention_mask=(attention_mask == 0).long() if attention_mask.dtype != torch.long else attention_mask
    print(attention_mask)

    with torch.no_grad():
        outputs = difmeasure_model(
            input_ids=input_ids,
            attention_mask = attention_mask
            #attention_mask = torch.where(attention_mask==0,1, float(0.0))
            
        )
        pred_scores = outputs.logits.squeeze(-1).cpu().numpy()  # shape: (batch,)
        print(pred_scores)
     

    gold_scores = labels.cpu().numpy()
    mse_errors = (pred_scores - gold_scores) ** 2

    # For each sample in batch, collect results
    for i in range(len(gold_scores)):
        result = {
            "gold": gold_scores[i],
            "pred": pred_scores[i],
            "mse_error": mse_errors[i],
        }
        if idxs is not None:
            result["idx"] = idxs[i].item() if hasattr(idxs[i], "item") else idxs[i]
        # Optionally add input_ids, sentence1, sentence2 if available
        results.append(result)
        print(results)
    # print(results[-1])  # Print last result for verification

df = pd.DataFrame(results)


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
[1.0501425 1.4354967]
[{'gold': 5.0, 'pred': 1.0501425, 'mse_error': 15.601374}]
[{'gold': 5.0, 'pred': 1.0501425, 'mse_e

KeyboardInterrupt: 

In [140]:
# Check proportion of validation samples exceeding 128 tokens
long_count = 0
total_count = 0
for example in stsb_train_dataset:
    if 0 not in example["input_ids"]:
        long_count += 1
    total_count += 1
print(f"Proportion exceeding 128 tokens: {long_count}/{total_count} = {long_count/total_count:.4f}")

Proportion exceeding 128 tokens: 0/5749 = 0.0000


In [45]:
import torch.nn as nn
neobert_model = mdl.NeoBERTLMHead(config)
neobert_backbone = neobert_model.model #ignore the LM head
from peft import LoraConfig, get_peft_model, TaskType
class NeoBERTBackboneWrapper(nn.Module):
    """
    Wraps NeoBERT backbone to accept HuggingFace-style arguments for LoRA compatibility.
    """
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, output_router_logits=True, output_expert_usage_loss=False, **kwargs):
        # Map input_ids -> src, attention_mask -> pad_mask
        src = input_ids
        pad_mask = attention_mask
        # Pass through to NeoBERT
        return self.backbone(src=src, pad_mask=pad_mask, output_router_logits=output_router_logits, output_expert_usage_loss=output_expert_usage_loss)


class NeoBERTForSTSB(nn.Module):
    def __init__(self, hidden_size, dropout_prob=0.1):
        super().__init__()
        # Wrap backbone for LoRA compatibility
        self.neobert_backbone = NeoBERTBackboneWrapper(neobert_backbone)
        self.dropout = nn.Dropout(dropout_prob)
        self.regression_head = nn.Linear(hidden_size, 1)  # single float output
        self.config = neobert_backbone.config # not quite same as cfg since its a NeoBERTconfig but we could equivalently have used cfg
        self.expert_dims = [int(expert_size) for expert_size in self.config.expert_sizes.split(",")]

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        backbone_outputs = self.neobert_backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_expert_usage_loss=True
        )

        # print("input_ids:", input_ids)
        # print("attention_mask:", attention_mask)
        # print("labels:", labels)

        final_hidden_state = backbone_outputs["hidden_representation"]
        router_logits  = backbone_outputs["router_logits"]

        pooled_output = final_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        # # print("pooled_output:", pooled_output)
        # print(self.regression_head)
        
        # Print all regression_head parameters
        # print("regression_head parameters:")
        # for name, param in self.regression_head.named_parameters():
        #     print(f"{name}: {param.data}")

    # # Print min and max of regression_head parameters
    #     params = list(self.regression_head.parameters())
    #     if params:
    #         weights = params[0]
    #         bias = params[1] if len(params) > 1 else None
    #         print("regression_head weights min:", weights.min().item(), "max:", weights.max().item())
    #         if bias is not None:
    #             print("regression_head bias min:", bias.min().item(), "max:", bias.max().item())

        logits = self.regression_head(pooled_output).squeeze(-1)
        # print("logits:", logits)

        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(logits.float(), labels.float())

        expert_usage_loss = self._MoEBlockRoutingCost(router_logits, attention_mask) #(batch_size,)

        return {"loss": loss, "logits": logits, "expert_usage_loss": expert_usage_loss}

    def _MoEBlockRoutingCost(self,
                            router_logits, #tuple of n_layers of tensors of size (batch * sequence_length, n_experts)
                            attention_mask #shape (batch,seq_length)
                            ):

        # Correct stacking of tuple of raw_routing_weights
        print("router logits shape:", router_logits[0].shape)
        
        router_logits = torch.stack(list(router_logits), dim=0) #shape (n_layers, batch * sequence_length, n_experts)
        raw_routing_weights = torch.softmax(router_logits, dim=-1)
        print("raw routing weights shape:", raw_routing_weights.shape)

        if len(self.expert_dims) > 1:
            expert_dims_tensor = torch.tensor(self.expert_dims,dtype=torch.float32,
                device=raw_routing_weights.device )
            normalisation_factor = torch.sum(expert_dims_tensor)
            normalised_expert_sizes = expert_dims_tensor / normalisation_factor#prevent overflow
            routing_costs = normalised_expert_sizes**self.config.expert_cost_exponent
            routing_costs = routing_costs.to(dtype = raw_routing_weights.dtype)#cast back to original dtype
            print("routing costs shape:", routing_costs.shape)

            n_layers,_,n_experts = raw_routing_weights.shape
            batch_size, seq_length = attention_mask.shape
            raw_routing_weights = raw_routing_weights.reshape(n_layers, batch_size, seq_length, n_experts)
            number_of_tokens_per_seq = seq_length
            

            #ignore padding_tokens
            if attention_mask!= None: #attention_mask has shape (Batch,seq_length) and routing_weights has shape (n_layers, batch * sequence_length, n_experts)
                multiplicative_mask = torch.where(attention_mask == 0, 1.0, 0.0).to(dtype = raw_routing_weights.dtype).reshape(1, batch_size, seq_length, 1)
                number_of_tokens_per_seq = multiplicative_mask.sum(dim=2).squeeze()
                print("number of tokens per seq:", number_of_tokens_per_seq)

                raw_routing_weights = raw_routing_weights * multiplicative_mask
                print("raw routing weights after masking shape:", raw_routing_weights.shape)
                print("raw routing weights after masking shape:", raw_routing_weights)
            # Compute usage loss per token
            expert_usage_loss = torch.einsum('mijk,k->mij', raw_routing_weights, routing_costs) #(n_layers,batch_size,seq_length)
            
            print("expert usage loss per token shape:", expert_usage_loss.shape)
            #sum across layers
            expert_usage_loss = torch.sum(expert_usage_loss, dim=0) #shape (batch_size,seq_length)
            print("expert usage loss:", expert_usage_loss)


            # Average over all tokens in the sequence
            normalised_mean_expert_usage_loss = torch.sum(expert_usage_loss, dim = 1) / number_of_tokens_per_seq #shape (batch_size,)
        else:
            normalised_mean_expert_usage_loss = None
            
        print("normalised mean expert usage loss shape:", normalised_mean_expert_usage_loss.shape)
        return normalised_mean_expert_usage_loss # the normalisation is not an issue as we care about comparisons but just a reminder it is here

    

neobert_stsb_model = NeoBERTForSTSB(
    hidden_size=config.hidden_size,
    dropout_prob=0.1
)







# lora_config = LoraConfig(
#     r=8,
#     lora_alpha=32,
#     target_modules=["qkv", "wo","in_proj", "out_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.FEATURE_EXTRACTION,  # <— avoids labels
#     )


#     # 2. Wrap your predictor model with LoRA
# for param in neobert_stsb_model.neobert_backbone.parameters():
#     param.requires_grad = False

# neobert_stsb_model.neobert_backbone = get_peft_model(neobert_stsb_model.neobert_backbone, lora_config)

In [46]:
def to_target_batch_size(
    batch,
    stored_batch ,
    target_size: int = 8,
    ):
    tmp = {}
    batch_size = batch["input_ids"].shape[0]

    # If the batch is toO large, we store samples
    if batch_size > target_size:
        for key in batch.keys():
            tmp[key] = torch.split(batch[key], [target_size, batch_size - target_size], dim=0)
            batch[key] = tmp[key][0]
            stored_batch[key] = tmp[key][1] if stored_batch[key] is None else torch.cat([tmp[key][1], stored_batch[key]], dim=0)

    # If the batch is to small, we fetch stored samples
    elif batch_size < target_size and stored_batch["input_ids"] is not None:
        stored_batch_size = stored_batch["input_ids"].shape[0]
        missing = target_size - batch_size

        # Fetch only necessary samples if storage is larger than required
        if missing < stored_batch_size:
            for key in batch.keys():
                stored_batch[key].to(batch[key].device)
                tmp[key] = torch.split(stored_batch[key], [missing, stored_batch_size - missing], dim=0)
                batch[key] = torch.cat([batch[key], tmp[key][0]], dim=0)
                stored_batch[key] = tmp[key][1]
                stored_batch[key].to("cpu", non_blocking=True)

        # Concatenate otherwise
        else:
            for key in batch.keys():
                batch[key] = torch.cat([batch[key], stored_batch[key]], dim=0)
                stored_batch[key] = None

    return batch, stored_batch


compiled_neobert_stsb_model = torch.compile(neobert_stsb_model)
stored_batch = {
    "input_ids": None,
    "attention_mask": None,
    "labels": None,
}

count = 1
for batch in train_dataloader:

    if batch["input_ids"].shape[0] != batch_size:
        batch, stored_batch = to_target_batch_size(batch, stored_batch, batch_size)
    if batch["input_ids"].shape[0] < batch_size:
        stored_batch = batch
        continue
    
    # Forward pass
    outputs = compiled_neobert_stsb_model(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        labels=batch["labels"]
    )
    loss = outputs["loss"]  # This is already MSE loss from your model's forward()
    print("Loss:", loss.item())
    print("Expert usage loss:", outputs["expert_usage_loss"])
    count += 1
    if count > 2:
        break

W0823 21:33:21.391000 67272 Lib\site-packages\torch\_dynamo\convert_frame.py:1125] WON'T CONVERT torch_dynamo_resume_in_forward_at_32 C:\Users\gruau\AppData\Local\Temp\ipykernel_67272\1552834711.py line 32 
W0823 21:33:21.391000 67272 Lib\site-packages\torch\_dynamo\convert_frame.py:1125] due to: 
W0823 21:33:21.391000 67272 Lib\site-packages\torch\_dynamo\convert_frame.py:1125] Traceback (most recent call last):
W0823 21:33:21.391000 67272 Lib\site-packages\torch\_dynamo\convert_frame.py:1125]   File "c:\Users\gruau\OneDrive\Documents\CentraleSupelec\3A\Stage Oxford\Implementation\NeoBERT\NeoBERTenv\Lib\site-packages\torch\_inductor\cpp_builder.py", line 130, in check_compiler_exist_windows
W0823 21:33:21.391000 67272 Lib\site-packages\torch\_dynamo\convert_frame.py:1125]     subprocess.check_output([compiler, "/help"], stderr=subprocess.STDOUT)
W0823 21:33:21.391000 67272 Lib\site-packages\torch\_dynamo\convert_frame.py:1125]   File "C:\Users\gruau\anaconda3\Lib\subprocess.py", line 

router logits shape: torch.Size([256, 8])
raw routing weights shape: torch.Size([4, 256, 8])
routing costs shape: torch.Size([8])
number of tokens per seq: tensor([16., 18.])
raw routing weights after masking shape: torch.Size([4, 2, 128, 8])
raw routing weights after masking shape: tensor([[[[0.1211, 0.1335, 0.1303,  ..., 0.1242, 0.1222, 0.1211],
          [0.1295, 0.1284, 0.1280,  ..., 0.1269, 0.1230, 0.1145],
          [0.1230, 0.1227, 0.1252,  ..., 0.1191, 0.1300, 0.1293],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.1209, 0.1335, 0.1301,  ..., 0.1241, 0.1226, 0.1214],
          [0.1294, 0.1283, 0.1278,  ..., 0.1268, 0.1234, 0.1147],
          [0.1242, 0.1314, 0.1313,  ..., 0.1201, 0.1247, 0.1162],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.000

In [ ]:
batch_size = 8
seq_len = 128
a = torch.randn(batch_size, seq_len)
b = torch.randn(batch_size, seq_len)
dummy_input = (a, b)
_MoEBlockRoutingCost()



In [58]:
def rewrite_inputs_with_tokenizer(batch, tokenizer):
    """
    Takes tokenized inputs and rewrites/detokenizes them back to text using the tokenizer.
    
    Args:
        batch: Dictionary containing 'input_ids' and optionally 'attention_mask'
        tokenizer: The tokenizer used for encoding/decoding
    
    Returns:
        Dictionary with original inputs and rewritten text
    """
    input_ids = batch["input_ids"]
    attention_mask = batch.get("attention_mask", None)
    
    results = []
    
    for i in range(input_ids.shape[0]):
        # Get individual sequence
        sequence_ids = input_ids[i]
        
        # If attention mask exists, use it to identify valid tokens
        if attention_mask is not None:
            # Convert additive mask back to standard mask for processing
            if attention_mask.dtype == torch.float32:
                # Additive mask: 0.0 for valid tokens, -inf for padding
                valid_mask = (attention_mask[i] != float('-inf'))
            else:
                # Standard mask: 1 for valid tokens, 0 for padding
                valid_mask = (attention_mask[i] == 1)
            
            # Only keep valid tokens
            valid_ids = sequence_ids[valid_mask]
        else:
            valid_ids = sequence_ids
        
        # Decode the sequence back to text
        decoded_text = tokenizer.decode(valid_ids, skip_special_tokens=True)
        
        # Store results
        results.append({
            'original_ids': sequence_ids.tolist(),
            'valid_ids': valid_ids.tolist(),
            'rewritten_text': decoded_text
        })
    
    return results

# Example usage with the existing batch
print("Testing rewrite function with STSB tokenizer:")
example_results = rewrite_inputs_with_tokenizer(batch, tokenizer_text_attack)

for i, result in enumerate(example_results):
    print(f"\nSample {i+1}:")
    print(f"Original token IDs: {result['original_ids'][:20]}...")  # Show first 20 tokens
    print(f"Valid token IDs: {result['valid_ids'][:20]}...")
    print(f"Rewritten text: {result['rewritten_text']}")

Testing rewrite function with STSB tokenizer:

Sample 1:
Original token IDs: [101, 1037, 2158, 3564, 2006, 1996, 2723, 3248, 1037, 2858, 1012, 102, 1037, 2158, 3564, 2006, 1996, 2723, 1999, 1037]...
Valid token IDs: [101, 1037, 2158, 3564, 2006, 1996, 2723, 3248, 1037, 2858, 1012, 102, 1037, 2158, 3564, 2006, 1996, 2723, 1999, 1037]...
Rewritten text: a man sitting on the floor plays a guitar. a man sitting on the floor in a room is strumming a guitar.

Sample 2:
Original token IDs: [101, 1037, 17022, 3248, 2007, 1037, 6081, 11661, 1012, 102, 1996, 3899, 2003, 2652, 2007, 1037, 6081, 11661, 1012, 102]...
Valid token IDs: [101, 1037, 17022, 3248, 2007, 1037, 6081, 11661, 1012, 102, 1996, 3899, 2003, 2652, 2007, 1037, 6081, 11661, 1012, 102]...
Rewritten text: a puppy plays with a plastic container. the dog is playing with a plastic container.


In [6]:
from typing import Union, Optional, Dict, Any
def get_entropy(gate_logits: Union[torch.Tensor, tuple[torch.Tensor], None], cfg, attention_mask):

    """
    Computes the entropy loss for the Heterogeneous Mixture of Experts (HMoE) model.

    Args:
        gate_logits: Logits from the gate, should be a tensor of shape [batch_size, sequence_length, num_experts].
        cfg: Configuration object containing model parameters.
        batch: The input batch containing labels and other necessary information.

    Returns:
        The computed entropy loss.
    """
    if isinstance(gate_logits, tuple):
        compute_device = gate_logits[0].device
        if attention_mask is None:
            concatenated_gate_logits = torch.cat([layer_gate.to(compute_device) for layer_gate in gate_logits], dim=0)#n_layers*batch_size*seq_length,num_expert
        else:
            concatenated_gate_logits = torch.stack([layer_gate.to(compute_device) for layer_gate in gate_logits], dim=0)#n_layers,batch_size*seq_length,num_experts
            boolean_pad_mask = (attention_mask.reshape(-1) != float("-inf")).squeeze(-1)#[batch_size*seq_length,num_experts]
            concatenated_gate_logits = concatenated_gate_logits[:,boolean_pad_mask,:].reshape(-1, concatenated_gate_logits.shape[-1]) #n_layers*batch_size*seq_length,num_expert

    else: 
        print("WARNING: gate_logits is not a tuple, this is not supported in the current implementation of entropy loss function. ")

    routing_weights = torch.nn.functional.softmax(concatenated_gate_logits, dim=-1)

    entropy_loss_epsilon = 1e-5
    
    # Compute the entropy
    entropy_loss = -torch.sum(routing_weights * torch.log(routing_weights + entropy_loss_epsilon), dim=-1)

    # Average
    entropy_loss = torch.mean(entropy_loss)

    return entropy_loss

In [16]:
n_layers = 2
n_seq = 4
n_batch = 3
n_experts = 8


In [25]:
# Test entropy calculation with uniform logits
import math

# Create uniform logits for 8 experts
n_experts = 8
n_tokens = 100  # arbitrary number of tokens

# Uniform logits (all zeros will give uniform probabilities after softmax)
uniform_logits = torch.zeros(n_tokens, n_experts)

# Calculate entropy manually
uniform_probs = torch.nn.functional.softmax(uniform_logits, dim=-1)
print("Uniform probabilities:", uniform_probs[0])  # Should be [0.125, 0.125, ...]

# Manual entropy calculation
manual_entropy = -torch.sum(uniform_probs * torch.log(uniform_probs + 1e-5), dim=-1)
print("Manual entropy per token:", manual_entropy[0])
print("Average manual entropy:", torch.mean(manual_entropy))

# Theoretical maximum entropy
theoretical_max = math.log(n_experts)
print("Theoretical maximum entropy:", theoretical_max)

# Test with your function
gate_logits = (uniform_logits,)  # Tuple format as expected by your function
attention_mask = None  # No masking

entropy_result = get_entropy(gate_logits, None, attention_mask)
print("Your function result:", entropy_result)

Uniform probabilities: tensor([0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250])
Manual entropy per token: tensor(2.0794)
Average manual entropy: tensor(2.0794)
Theoretical maximum entropy: 2.0794415416798357
Your function result: tensor(2.0794)


In [26]:
# Clarify the logarithm confusion
import math

print("=== LOGARITHM CLARIFICATION ===")
print(f"Natural log (ln) of 8: {math.log(8):.6f}")
print(f"Log base 2 of 8: {math.log2(8):.6f}")
print(f"Log base 10 of 8: {math.log10(8):.6f}")

print("\n=== ENTROPY UNITS ===")
print("- Natural log (ln): measured in 'nats'")
print("- Log base 2: measured in 'bits'") 
print("- Log base 10: measured in 'dits' or 'bans'")

print(f"\nFor uniform distribution over 8 experts:")
print(f"- Maximum entropy in nats: {math.log(8):.6f}")
print(f"- Maximum entropy in bits: {math.log2(8):.6f}")

print(f"\nPyTorch uses natural log by default:")
print(f"torch.log(8) = {torch.log(torch.tensor(8.0)).item():.6f}")

print(f"\nYour entropy result of ~2.08 is correct in NATS!")
print(f"If you want bits, convert: {math.log(8) / math.log(2):.6f} bits")

=== LOGARITHM CLARIFICATION ===
Natural log (ln) of 8: 2.079442
Log base 2 of 8: 3.000000
Log base 10 of 8: 0.903090

=== ENTROPY UNITS ===
- Natural log (ln): measured in 'nats'
- Log base 2: measured in 'bits'
- Log base 10: measured in 'dits' or 'bans'

For uniform distribution over 8 experts:
- Maximum entropy in nats: 2.079442
- Maximum entropy in bits: 3.000000

PyTorch uses natural log by default:
torch.log(8) = 2.079442

Your entropy result of ~2.08 is correct in NATS!
If you want bits, convert: 3.000000 bits


In [5]:
import numpy as np
experts = [0, 252, 308, 364, 420, 476]
mean_expert_usage= np.mean([(expert)**2 for expert in experts])
print(mean_expert_usage*12)


1387680.0
